![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

# Getting Started with Iceberg and Spark

In [1]:
spark

## Load One Month of NYC Taxi/Limousine Trip Data

For this notebook, we will use the New York City Taxi and Limousine Commision Trip Record Data that's available on the AWS Open Data Registry. This contains data of trips taken by taxis and for-hire vehicles in New York City. We'll save this into an iceberg table called `taxis`.

In [2]:
df = spark.read.option("header", True).csv("/home/iceberg/data/yellow_tripdata_2020-04.csv")
df.write.saveAsTable("taxis")
spark.sql("DESCRIBE TABLE taxis").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|            VendorID|   string|       |
|tpep_pickup_datetime|   string|       |
|tpep_dropoff_date...|   string|       |
|     passenger_count|   string|       |
|       trip_distance|   string|       |
|          RatecodeID|   string|       |
|  store_and_fwd_flag|   string|       |
|        PULocationID|   string|       |
|        DOLocationID|   string|       |
|        payment_type|   string|       |
|         fare_amount|   string|       |
|               extra|   string|       |
|             mta_tax|   string|       |
|          tip_amount|   string|       |
|        tolls_amount|   string|       |
|improvement_surch...|   string|       |
|        total_amount|   string|       |
|congestion_surcharge|   string|       |
|                    |         |       |
|      # Partitioning|         |       |
+--------------------+---------+-------+
only showing top

## Schema Evolution

Adding, dropping, renaming, or altering columns is easy and safe in Iceberg. In this example, we'll rename `fare_amount` to `fare` and `trip_distance` to `distance`. We'll also add a float column `fare_per_distance_unit` immediately after `distance`.

In [3]:
spark.sql("""
    ALTER TABLE taxis
    RENAME COLUMN fare_amount TO fare
""")
spark.sql("""
    ALTER TABLE taxis
    RENAME COLUMN trip_distance TO distance
""")
spark.sql("""
    ALTER TABLE taxis
    ADD COLUMN fare_per_distance_unit float AFTER distance
""")

DataFrame[]

Let's update the new `fare_per_distance_unit` to equal `fare` divided by `distance`.

In [4]:
spark.sql("""
UPDATE taxis
SET fare_per_distance_unit = fare/distance
""")
spark.sql("""
    SELECT
    VendorID
    ,tpep_pickup_datetime
    ,tpep_dropoff_datetime
    ,fare
    ,distance
    ,fare_per_distance_unit
    FROM taxis
""").show()

+--------+--------------------+---------------------+-----+--------+----------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime| fare|distance|fare_per_distance_unit|
+--------+--------------------+---------------------+-----+--------+----------------------+
|       1| 2020-04-01 00:41:22|  2020-04-01 01:01:53|  5.5|    1.20|             4.5833335|
|       1| 2020-04-01 00:56:00|  2020-04-01 01:09:25| 12.5|    3.40|             3.6764705|
|       1| 2020-04-01 00:00:26|  2020-04-01 00:09:25|   10|    2.80|             3.5714285|
|       1| 2020-04-01 00:24:38|  2020-04-01 00:34:38|   10|    2.60|             3.8461537|
|       2| 2020-04-01 00:13:24|  2020-04-01 00:18:26|  6.5|    1.44|              4.513889|
|       2| 2020-04-01 00:24:36|  2020-04-01 00:33:09| 10.5|    2.93|             3.5836177|
|       2| 2020-04-01 00:56:56|  2020-04-01 01:09:13|   20|    6.86|              2.915452|
|       2| 2020-04-01 00:06:56|  2020-04-01 00:14:15|    7|    1.19|            

## Row Level Deletes
With Iceberg tables, `DELETE` queries can be used to perform row-level deletes. This is as simple as providing the table name and a `WHERE` predicate. If the filter matches an entire partition of the table, Iceberg will intelligently perform a metadata-only operation where it simply deletes the metadata for that partition.

Let's perform a row-level delete for all rows that have a `fare_per_distance_unit` greater than 4 and a `distance` greater than 2. This should leave us with relatively short trips that have a relatively high fare per distance traveled.

In [5]:
spark.sql("""
    DELETE FROM taxis
    WHERE fare_per_distance_unit > 4.0
    OR distance > 2.0
""")

21/11/19 11:49:08 WARN RewriteDelete: Cannot translate expression to source filter: ((fare_per_distance_unit#304 > 4.0) OR (cast(distance#303 as double) > 2.0))


DataFrame[]

There are some fares that have a `null` for `fare_per_distance_unit` due to the distance being `0`. Let's remove those as well.

In [6]:
spark.sql("""
    DELETE FROM taxis
    WHERE fare_per_distance_unit is null
""")

DataFrame[]

In [7]:
spark.sql("""
    SELECT
    VendorID
    ,tpep_pickup_datetime
    ,tpep_dropoff_datetime
    ,fare
    ,distance
    ,fare_per_distance_unit
    FROM taxis
""").show()

+--------+--------------------+---------------------+----+--------+----------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|fare|distance|fare_per_distance_unit|
+--------+--------------------+---------------------+----+--------+----------------------+
|       2| 2020-04-01 00:42:56|  2020-04-01 00:48:41|  -7|    1.34|             -5.223881|
|       2| 2020-04-01 00:26:49|  2020-04-01 00:31:04| 6.5|    1.69|             3.8461537|
|       2| 2020-04-01 00:42:30|  2020-04-01 00:46:58| 6.5|    1.70|             3.8235295|
|       1| 2020-04-01 00:36:15|  2020-04-01 00:41:30|   7|    1.80|             3.8888888|
|       2| 2020-04-01 01:16:34|  2020-04-01 01:23:20|  -8|    1.85|             -4.324324|
|       2| 2020-04-01 01:51:36|  2020-04-01 01:57:59| 7.5|    1.88|             3.9893618|
|       1| 2020-04-01 02:13:27|  2020-04-01 02:16:30|   6|    1.50|                   4.0|
|       2| 2020-04-01 02:25:38|  2020-04-01 02:26:05|-2.5|     .08|                -31.25|

## Metadata Tables

Iceberg tables contain very rich metadata that can be easily queried. For example, you can retrieve the manifest list for any snapshot, simply by querying the table's `snapshots` table.

In [8]:
spark.sql("""
    SELECT snapshot_id, manifest_list
    FROM taxis.snapshots
""").show()

21/11/19 11:49:12 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
21/11/19 11:49:12 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
21/11/19 11:49:14 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
21/11/19 11:49:14 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore UNKNOWN@172.22.0.3


+-------------------+--------------------+
|        snapshot_id|       manifest_list|
+-------------------+--------------------+
|2698491583658191945|/home/iceberg/war...|
|6211071195367590713|/home/iceberg/war...|
|6548397803384183321|/home/iceberg/war...|
|3201604413922280521|/home/iceberg/war...|
+-------------------+--------------------+



21/11/19 11:49:14 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


The `files` table contains loads of information on data files, including column level statistics such as null counts, lower bounds, and upper bounds.

In [9]:
spark.sql("""
    SELECT file_path, file_format, record_count, null_value_counts, lower_bounds, upper_bounds
    FROM taxis.files
""").show()

+--------------------+-----------+------------+--------------------+--------------------+--------------------+
|           file_path|file_format|record_count|   null_value_counts|        lower_bounds|        upper_bounds|
+--------------------+-----------+------------+--------------------+--------------------+--------------------+
|/home/iceberg/war...|    PARQUET|        4953|{1 -> 53, 2 -> 0,...|{1 -> 1, 2 -> 202...|{1 -> 2, 2 -> 202...|
+--------------------+-----------+------------+--------------------+--------------------+--------------------+



## Time Travel

The history table lists all snapshots and which parent snapshot they derive from. The `is_current_ancestor` flag let's you know if a snapshot is part of the linear history of the current snapshot of the table.

In [10]:
spark.sql("""
SELECT *
FROM taxis.history
""").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2021-11-19 11:49:...|2698491583658191945|               null|               true|
|2021-11-19 11:49:...|6211071195367590713|2698491583658191945|               true|
|2021-11-19 11:49:...|6548397803384183321|6211071195367590713|               true|
|2021-11-19 11:49:...|3201604413922280521|6548397803384183321|               true|
+--------------------+-------------------+-------------------+-------------------+



You can time-travel by altering the `current-snapshot-id` property of the table to reference any snapshot in the table's history. Let's revert the table to it's original state by traveling to the very first snapshot ID.

In [11]:
df = spark.sql("""
SELECT *
FROM taxis.history
""")
original_snapshot = df.head().snapshot_id
print(original_snapshot)

2698491583658191945


In [12]:
spark.sql(f"""
    ALTER TABLE taxis
    SET TBLPROPERTIES ('current-snapshot-id'={original_snapshot})
""").show()
spark.sql("""
    SELECT
    VendorID
    ,tpep_pickup_datetime
    ,tpep_dropoff_datetime
    ,fare
    ,distance
    ,fare_per_distance_unit
    FROM taxis
""").show()

++
||
++
++

+--------+--------------------+---------------------+-----+--------+----------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime| fare|distance|fare_per_distance_unit|
+--------+--------------------+---------------------+-----+--------+----------------------+
|       1| 2020-04-01 00:41:22|  2020-04-01 01:01:53|  5.5|    1.20|                  null|
|       1| 2020-04-01 00:56:00|  2020-04-01 01:09:25| 12.5|    3.40|                  null|
|       1| 2020-04-01 00:00:26|  2020-04-01 00:09:25|   10|    2.80|                  null|
|       1| 2020-04-01 00:24:38|  2020-04-01 00:34:38|   10|    2.60|                  null|
|       2| 2020-04-01 00:13:24|  2020-04-01 00:18:26|  6.5|    1.44|                  null|
|       2| 2020-04-01 00:24:36|  2020-04-01 00:33:09| 10.5|    2.93|                  null|
|       2| 2020-04-01 00:56:56|  2020-04-01 01:09:13|   20|    6.86|                  null|
|       2| 2020-04-01 00:06:56|  2020-04-01 00:14:15|    7|    1.19

Another look at the history table shows that the original state of the table has been added as a new entry
with the original snapshot ID.

In [13]:
spark.sql("""
SELECT *
FROM taxis.history
""").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2021-11-19 11:49:...|2698491583658191945|               null|               true|
|2021-11-19 11:49:...|6211071195367590713|2698491583658191945|              false|
|2021-11-19 11:49:...|6548397803384183321|6211071195367590713|              false|
|2021-11-19 11:49:...|3201604413922280521|6548397803384183321|              false|
|2021-11-19 11:49:...|2698491583658191945|               null|               true|
+--------------------+-------------------+-------------------+-------------------+



In [14]:
spark.sql("""
    SELECT
    VendorID
    ,tpep_pickup_datetime
    ,tpep_dropoff_datetime
    ,fare
    ,distance
    ,fare_per_distance_unit
    FROM taxis
""").show()

+--------+--------------------+---------------------+-----+--------+----------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime| fare|distance|fare_per_distance_unit|
+--------+--------------------+---------------------+-----+--------+----------------------+
|       1| 2020-04-01 00:41:22|  2020-04-01 01:01:53|  5.5|    1.20|                  null|
|       1| 2020-04-01 00:56:00|  2020-04-01 01:09:25| 12.5|    3.40|                  null|
|       1| 2020-04-01 00:00:26|  2020-04-01 00:09:25|   10|    2.80|                  null|
|       1| 2020-04-01 00:24:38|  2020-04-01 00:34:38|   10|    2.60|                  null|
|       2| 2020-04-01 00:13:24|  2020-04-01 00:18:26|  6.5|    1.44|                  null|
|       2| 2020-04-01 00:24:36|  2020-04-01 00:33:09| 10.5|    2.93|                  null|
|       2| 2020-04-01 00:56:56|  2020-04-01 01:09:13|   20|    6.86|                  null|
|       2| 2020-04-01 00:06:56|  2020-04-01 00:14:15|    7|    1.19|            